In [1]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import pandas as pd

artist_list = pd.read_csv('/Users/erichochberger/Documents/GitHub/data-exploration/Dap Artists Pull - Dapioartists.csv')




In [7]:
list(artist_list['Artist Name'][1:100])

['The Vices',
 'The Strokes',
 'Moby',
 'Pixies',
 'Knxwledge',
 'Sen Senra',
 'Eric Prydz',
 'DZ Deathrays',
 'John Mellencamp',
 'Larry and His Flask',
 'Donny Benét',
 'Vickeblanka',
 'Scissor Sisters',
 'Landon McNamara',
 'Gamblers',
 'Afefe Iku',
 'August Charles',
 'TOOL',
 'Uffie',
 'AJ Tracey',
 'Joe Higgs',
 'Kari Parker',
 'Jean Dawson',
 "Dre'es",
 'Thinking Caps',
 'Drive-By Truckers',
 'Tedax Max',
 'Niro',
 'Joe Cocker',
 'Kay9ine',
 'Matuê',
 'Blakkamoore',
 'Revival',
 'Denzel Curry',
 'Rachael Lampa',
 'Thunder',
 'Madeline The Person',
 'Fumes',
 'PEEKABOO',
 'Lovebirds',
 'Earshot',
 'Evanescence',
 'Prince Bright',
 'Deadly Hunta',
 'Foxy Shazam',
 'Badi',
 'Regina Belle',
 'The Winans',
 'Luh Tyler',
 'Jahmali',
 'George Nooks',
 'Tinlicker',
 'Gladys Knight & The Pips',
 'Aluna',
 'BERWYN',
 'Ta-ku',
 'Chop Daily',
 'Montell Jordan',
 'Rachel Platten',
 'Jake Miller',
 'Blur',
 'C-Zar',
 'Keys N Krates',
 'Young Shanty',
 'Kizz Daniel',
 'Mark Mothersbaugh',
 'Is

In [4]:
df = []



for artist_name in list(artist_list['Artist Name'][1:7439]):
  search_url = 'https://www.songkick.com/search?page=1&per_page=10&query=' + artist_name + '&type=artists'

  response = requests.get(search_url)
  soup = BeautifulSoup(response.content, "html.parser")

  try:
    artist_query = soup.find_all(class_ = "summary")[0].find_all('a', href=True)[0]['href']
  except:
    continue

    # with open("tennis.csv", "w", newline="") as csv_file:
#   # Create a CSV writer
#   writer = csv.writer(csv_file)
#   # Write the header row
#   writer.writerow(["artist", "date", "venue", "city", "region", "country", "performers"])


  # Set the initial page number
  page = 1

  # Set a flag to indicate if we should continue scraping
  keep_scraping = True

  # Start the scraping loop
  while keep_scraping:
    # Construct the URL for the current page
    url = "https://www.songkick.com" + artist_query + "/gigography?page=" + str(page) 

    # Make a request to the URL
    response = requests.get(url, allow_redirects=False)

    # Check if the request was successful
    if response.status_code != 200:
      print("Failed to make request")
      exit()

    # Parse the HTML of the page
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all the div elements with the class 'microformat'
    concerts = soup.find_all("div", class_="microformat")

    # Iterate over the concerts
    for concert in concerts:
      # Parse the JSON script
      data = json.loads("".join(concert.find("script", {"type":"application/ld+json"})))[0]
      # print(data, "\n\n")

      # Check if the location @type is "Place"
      if data["location"]["@type"] == "Place":
        # Extract the necessary information
        artist = data["name"]
        city = data["location"]["address"]["addressLocality"]
        country = data["location"]["address"]["addressCountry"]
        date = data["startDate"]
        venue = data["location"]["name"]
        # Check if the address region key exists (USA AND CANADA ONLY)
        region = ""
        if "addressRegion" in data["location"]["address"]:
            region = data["location"]["address"]["addressRegion"]
        # Get performer data and cycle through it getting names
        performers = []
        for performer in data["performer"]:
            performers.append(performer["name"])

        # Write the information to the CSV file as a new record
        # writer.writerow([artist, date, venue, city, region, country, performers])
        df.append({'artist' : artist, 'date' : date, 'venue' : venue, 'city' : city, 'region' : region, 'country' : country, 'performers' : performers})

    # Check if there are more pages to scrape
    if len(concerts) < 50:
      # If there are less than 50 concerts on the page, it means we have reached the end of the gigography
      keep_scraping = False
    else:
      # If there are 50 concerts on the page, there may be more pages to scrape
      page += 1



In [15]:
pd.DataFrame(df)

,artist,date,venue,city,region,country,performers
0,The Vices (official),2023-01-21T14:30:00+0100,PlatoSonic,Groningen,,Netherlands,[The Vices (official)]
1,The Vices (official),2023-01-18T10:00:00+0100,Eurosonic,Groningen,,Netherlands,[The Vices (official)]
2,The Vices (official),2022-12-31,Groningen Telt Af 2022,Groningen,,Netherlands,[The Vices (official)]
3,The Vices (official),2022-11-17T19:30:00+0100,Effenaar,Eindhoven,,Netherlands,[The Vices (official)]
4,The Vices (official),2022-10-29,Rust,Copenhagen,,Denmark,[The Vices (official)]
...,...,...,...,...,...,...,...
120,The Vices (official),2019-03-06T20:00:00+0000,Dusk,York,,UK,[The Vices (official)]
121,The Vices (official),2019-03-05,West Street Live,Sheffield,,UK,[The Vices (official)]
122,The Vices (official),2019-03-03,Tyne Bank Brewery,Newcastle Upon Tyne,,UK,[The Vices (official)]
123,The Vices (official),2019-03-01,The White Hart,Corby,,UK,[The Vices (official)]


In [9]:
search_url = 'https://www.songkick.com/search?page=1&per_page=10&query=' + artist_list['Artist Name'][1] + '&type=artists'

response = requests.get(search_url)
soup = BeautifulSoup(response.content, "html.parser")


artist_query = soup.find_all(class_ = "summary")[0].find_all('a', href=True)[0]['href']

In [7]:
len(df['venue'].unique())

3821